In [321]:
import numpy as np
import matplotlib.pyplot as plt
import pylcp
from pylcp.atom import atom
import scipy.constants as cts
import magpylib as magpy
from scipy.stats import rv_continuous
import random

In [ ]:
tfmot = magpy.Collection()
N = 0
angles = [0,90,180,270]
angles_d = [45,225,225,45]
l = 5
p = [0,0,0,0,20,20,20,20]
s  = [-1,-1,1,1,1,1,-1,-1]
H = 40
h = [-H,-H,-H,-H,H,H,H,H]
mstyle = dict(
    mode="color+arrow",
    color=dict(north="magenta", middle="white", south="turquoise"),
    arrow=dict(width=2, color="k")
)
for a in range(len(angles)):
    cube= magpy.magnet.Cuboid(
    dimension=(6,8,80),
    polarization=(s[a]*1.42,0,0),
    position=(h[a],h[a],h[a]),
    style_magnetization=mstyle )
    
    cube.rotate_from_angax(angles[a], 'z',anchor = 0)
    cube.rotate_from_angax(angles_d[a], 'z')

    tfmot.add(cube)

magpy.show(tfmot.rotate_from_angax(45,'z'), backend='plotly')

In [ ]:
# Continuation from above - ensure previous code is executed

import matplotlib.pyplot as plt
fig, ax = plt.subplots()

# Compute and plot field on x-y grid
grid = np.mgrid[-30:30:100j, -30:30:100j, 0:0:1j].T[0]
X, Y, _ = np.moveaxis(grid, 2, 0)

B = tfmot.getB(grid)*1e3
Bx, By, _ = np.moveaxis(B, 2, 0)
Bamp = np.linalg.norm(B, axis=2)

pc = ax.contourf(X, Y, Bamp, levels=50, cmap="coolwarm")
ax.streamplot(X, Y, Bx, By, color="k", density=1.5, linewidth=1)

# Add colorbar
fig.colorbar(pc, ax=ax, label="|B| (mT)")

# Figure styling
ax.set(
    xlabel="x-position(mm)",
    ylabel="y-position(mm)",
    aspect=1,
)

plt.show()

In [ ]:
rb87 = atom('87Rb')

klab = 2*np.pi*rb87.transition[1].k # Lab wavevector (without 2pi) in cm^{-1}
taulab = rb87.state[2].tau  # Lifetime of 6P_{3/2} state (in seconds)
gammalab = 1/taulab
Blab = 15 # About 15 G/cm is a typical gradient for Rb
mass_lab = 173*cts.value('atomic mass constant')
print(klab, taulab, gammalab/2/np.pi)

In [ ]:
# Now, here are our `natural' length and time scales:
x0 = 1/klab  # cm
t0 = taulab  # s
vc = x0/t0*(1/100)
Fc = gammalab*cts.hbar*klab*100


mass = 87*cts.value('atomic mass constant')*(x0*1e-2)**2/cts.hbar/t0

# And now our wavevector, decay rate, and magnetic field gradient in these units:
k = klab*x0
gamma = gammalab*t0
alpha = cts.value('Bohr magneton')*1e-4*5*x0*t0/cts.hbar

print(x0, t0, mass, k, gamma, alpha)

In [ ]:
1e500 - 1e500

In [ ]:
H_g_D2, mu_q_g_D2 = pylcp.hamiltonians.hyperfine_coupled(
0, 5/2, 0,  	-0.2592,
    Ahfs = 0, Bhfs=0, Chfs=0,
    muB=1)# ground state 1s0
H_e_D2, mu_q_e_D2 = pylcp.hamiltonians.hyperfine_coupled(
1, 5/2, 1.035,	-0.2592,
    Ahfs=59.52*1e6,Bhfs = 601.87*1e6 , Chfs= 0,
    muB=1) #excited state 1p1

dijq_D2 = pylcp.hamiltonians.dqij_two_hyperfine_manifolds(0, 1, 5/2)

E_e_D2 = np.unique(np.diagonal(H_e_D2))
E_g_D2 = np.unique(np.diagonal(H_g_D2))
hamiltonian_D2 = pylcp.hamiltonian(mass = mass)
hamiltonian_D2.add_H_0_block('g', H_g_D2)
hamiltonian_D2.add_H_0_block('e', H_e_D2)
hamiltonian_D2.add_d_q_block('g', 'e', dijq_D2, gamma = gamma, k = k)
hamiltonian_D2.add_mu_q_block('g', mu_q_g_D2)
hamiltonian_D2.add_mu_q_block('e', mu_q_e_D2)



dijq = pylcp.hamiltonians.dqij_two_bare_hyperfine(0, 1)




det = -15
s = 100

# Define the laser beams:
laserBeams = pylcp.laserBeams(
    [{'kvec':k*np.array([ 1.,  0., 0.]), 's':s, 'pol':-1, 'delta': E_e_D2[2] - E_g_D2[0] +  det*gamma},
     {'kvec':k*np.array([-1.,  0., 0.]), 's':s, 'pol':-1, 'delta': E_e_D2[2] - E_g_D2[0] +  det*gamma},
     {'kvec':k*np.array([ 0.,  1,  0.]), 's':s, 'pol':1, 'delta': E_e_D2[2] - E_g_D2[0] +  det*gamma},
     {'kvec':k*np.array([ 0., -1., 0.]), 's':s, 'pol':1, 'delta': E_e_D2[2] - E_g_D2[0] +  det*gamma}],
beam_type= pylcp.infinitePlaneWaveBeam
)

# Define the magnetic field:
print(alpha)
bf_mot = lambda R:tfmot.getB(R*10*x0)*cts.value('Bohr magneton')*t0/cts.hbar 
linGrad = pylcp.magField(lambda R: bf_mot(R))
#linGrad = pylcp.magField(lambda R: -alpha*R)

In [456]:
rateeq = pylcp.rateeq(laserBeams, linGrad, hamiltonian_D2, include_mag_forces=True)

In [ ]:
x = np.arange(-10, 10, 0.04)/x0
v = np.arange(-14, 14, 0.4)/vc

X, V = np.meshgrid(x, v)

Rvec = np.array([ np.zeros(X.shape),X, np.zeros(X.shape)])
Vvec = np.array([np.zeros(V.shape),V, np.zeros(V.shape)])

rateeq.generate_force_profile(Rvec, Vvec, name='Fx', progress_bar=True)

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, num="Expression", figsize=(6.5, 2.75))
im1 = ax.contourf(np.array(x)*x0, np.array(v)*vc, rateeq.profile['Fx'].F[1]*Fc)
fig.subplots_adjust(left=0.08, wspace=0.2)
cb1 = plt.colorbar(im1)
cb1.set_label('$F(N)$')
ax.set_xlabel('$x$ (cm)')
ax.set_ylabel('$v (ms^{-1})$')
ax.set_title('Force profile (with real magnetic field)')


In [434]:
# Constants
k_B = cts.k  # Boltzmann constant in J/K
 # Atomic mass of Yb-173
mass_kg = mass_lab  # Convert to kilograms
temperature = 405 + 273.15  # Convert Celsius to Kelvin

# Velocity Probability Function
def p(m, T, v):
    return (m / (k_B * T))**2 * np.exp(-m * v**2 / (2 * k_B * T)) * (v**3) / 2

In [435]:
class MaxwellBoltzmann(rv_continuous):
    def _pdf(self, v):
        return p(mass_kg, temperature, v)

# Define the custom distribution
maxwell_boltzmann = MaxwellBoltzmann(a=0, name="Maxwell-Boltzmann")

In [ ]:
(t_span[1]**2)*1e-30/mass - (t_span[0]**2)*1e-30/mass

In [ ]:
size = 10000# Number of samples
velocities = maxwell_boltzmann.rvs(size=size)
velocities = velocities[velocities < 400]
random_numbers =  [random.uniform(0,35e-3) for i in range(size)]
velocities_t = [np.tan(random_numbers[i])*velocities[i] for i in range(len(velocities)) ]
v_0 = []
for i in range(len(velocities)):
    v_0.append([velocities_t[i],velocities_t[i],50])
# Display the sampled velocities
print(len(velocities))
plt.hist(velocities, bins=100, density=True)
plt.grid()

In [ ]:
v_0[11]

In [ ]:
v_0[11]

In [440]:
def simulate_trajectory(v_0, t_span):

    x_t, y_t, v_tx, v_ty, t = [], [], [], [], []

    di = 0  # Initial time step
    s = np.array([0.5/x0, 0.5/x0, 0.5/x0])  
    b = 0  # Step counter

    for q in range(len(t_span)):
        v_x, v_y, v_z = v_0  

        dt = (t_span[q] - di) / t0  

   
        rateeq.generate_force_profile(s,v_0 , name='Fz', progress_bar=False)
        

        # Acceleration (force divided by mass)
        a_x = rateeq.profile['Fz'].F[0] / mass
        a_y = rateeq.profile['Fz'].F[1] / mass
        a_z = 0  

        # Update position
        s_x = (1 / 2) * a_x * (dt ** 2) + v_x * dt + s[0]
        s_y = (1 / 2) * a_y * (dt ** 2) + v_y * dt + s[1]
        s_z = (1 / 2) * a_z * (dt ** 2) + v_z * dt + s[2]

        # Update velocity
        v_x += a_x * dt
        v_y += a_y * dt
        v_z += a_z * dt

        # Append values
        x_t.append(s_x)
        y_t.append(s_y)
        v_tx.append(v_x)
        v_ty.append(v_y)
        t.append(t_span[q])

        # Update initial conditions for the next step
        di = t_span[q]
        v_0 = [v_x, v_y, v_z]
        s = np.array([s_x, s_y, s_z])

        # Stop condition
        if s_z * x0 > 0.8:  # Stop if z-position exceeds threshold
            break

    return x_t, y_t, v_tx, v_ty, t


In [ ]:
x_n = []
y_n = []
v_tx_n = []
v_ty_n = []
t_n = []
t_max = 0.05
t_span = np.linspace(0, t_max, 10000)
v_01 = [v_0[0]]
b = 0
for i in v_0:
    x_t, y_t, v_tx, v_ty, t = simulate_trajectory(np.array(i)/vc, t_span)

    x_n.append(x_t)
    y_n.append(y_t)
    v_tx_n.append(v_tx)
    v_ty_n.append(v_ty)
    t_n.append(t)

    b += 1
    print(b)

In [463]:
intial_vel = []
final_vel = []
for i in v_tx_n:
 intial_vel.append(i[0])
 final_vel.append(i[-1])

In [ ]:
len(intial_vel)

In [473]:
intial_vel_x = [i[0] for i in v_tx_n]
final_vel_x = [i[-1] for i in v_tx_n]
intial_vel_y = [i[0] for i in v_ty_n]
final_vel_y = [i[-1] for i in v_ty_n]
final_posi_X = [i[-1]*x0 for i in x_n]
intial_posi_X = [i[0]*x0 for i in x_n]
final_posi_Y = [i[-1]*x0 for i in y_n]
intial_posi_Y = [i[0]*x0 for i in y_n]



In [ ]:
#plt.scatter(intial_posi_X,intial_posi_Y)
#plt.scatter(final_posi_X,final_posi_Y)

In [ ]:
#x = np.linspace(0, 14, 100)
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].plot(x,x)
ax[1].plot(x,x) 
ax[0].scatter(np.array(intial_vel_x)*vc,np.array(final_vel_x)*vc)
ax[1].scatter(np.array(intial_vel_y)*vc,np.array(final_vel_y)*vc)
ax[0].set_xlabel('Initial $velocity_x$(m/s)')
ax[0].set_ylabel('$Final velocity_x$(m/s)')
ax[1].set_xlabel('$Initial velocity_y$ (m/s)')
ax[1].set_ylabel('$Final velocity_y$ (m/s)')

plt.suptitle('Scatter plot of initial and final velocities for 2D MOT(x,y direction)')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,5))
for i in range(len(v_tx_n)):
    ax[0].plot(np.array(t_n[i])*t0, np.array(v_tx_n[i])*vc)
    ax[1].plot(np.array(t_n[i])*t0, np.array(x_n[i])*x0)
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Velocity (m/s)')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Position (cm)')